# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 8 2022 12:27PM,250315,10,CTF0011497,"Citieffe, Inc.",Released
1,Nov 8 2022 12:27PM,250315,10,CTF0011498,"Citieffe, Inc.",Released
2,Nov 8 2022 12:25PM,250314,16,8749677,Sartorius Bioprocess Solutions,Released
3,Nov 8 2022 12:12PM,250308,10,PRF-41213,Bio-PRF,Released
4,Nov 8 2022 12:12PM,250308,10,PRF-41217,Bio-PRF,Released
5,Nov 8 2022 12:12PM,250308,10,PRF-41227,Bio-PRF,Released
6,Nov 8 2022 12:12PM,250308,10,PRF-41231,Bio-PRF,Released
7,Nov 8 2022 12:12PM,250308,10,PRF-41234,Bio-PRF,Released
8,Nov 8 2022 12:12PM,250308,10,PRF-41238,Bio-PRF,Released
9,Nov 8 2022 12:12PM,250308,10,PRF-41241,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,250310,Released,20
41,250311,Released,1
42,250313,Released,1
43,250314,Released,1
44,250315,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
250310,NaN,NaN,1.0,20.0
250311,NaN,NaN,NaN,1.0
250313,NaN,NaN,NaN,1.0
250314,NaN,NaN,NaN,1.0
250315,NaN,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
250136,0.0,0.0,0.0,24.0
250194,0.0,0.0,89.0,16.0
250204,0.0,0.0,0.0,1.0
250206,0.0,0.0,0.0,1.0
250235,0.0,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
250136,0.0,0,0,24
250194,0.0,0,89,16
250204,0.0,0,0,1
250206,0.0,0,0,1
250235,0.0,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,250136,0.0,0,0,24
1,250194,0.0,0,89,16
2,250204,0.0,0,0,1
3,250206,0.0,0,0,1
4,250235,0.0,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,250136,0.0,,,24
1,250194,0.0,,89,16
2,250204,0.0,,,1
3,250206,0.0,,,1
4,250235,0.0,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 8 2022 12:27PM,250315,10,"Citieffe, Inc."
2,Nov 8 2022 12:25PM,250314,16,Sartorius Bioprocess Solutions
3,Nov 8 2022 12:12PM,250308,10,Bio-PRF
14,Nov 8 2022 12:07PM,250313,10,Emerginnova
15,Nov 8 2022 12:07PM,250310,19,"AdvaGen Pharma, Ltd"
36,Nov 8 2022 12:04PM,250311,16,TASA USA Inc.
37,Nov 8 2022 11:56AM,250280,15,"Alliance Pharma, Inc."
81,Nov 8 2022 11:46AM,250307,10,"Nextsource Biotechnology, LLC"
83,Nov 8 2022 11:46AM,250306,10,ISDIN Corporation
105,Nov 8 2022 11:44AM,250305,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Nov 8 2022 12:27PM,250315,10,"Citieffe, Inc.",0.0,,,2
1,Nov 8 2022 12:25PM,250314,16,Sartorius Bioprocess Solutions,0.0,,,1
2,Nov 8 2022 12:12PM,250308,10,Bio-PRF,0.0,,,11
3,Nov 8 2022 12:07PM,250313,10,Emerginnova,0.0,,,1
4,Nov 8 2022 12:07PM,250310,19,"AdvaGen Pharma, Ltd",0.0,,1,20
5,Nov 8 2022 12:04PM,250311,16,TASA USA Inc.,0.0,,,1
6,Nov 8 2022 11:56AM,250280,15,"Alliance Pharma, Inc.",0.0,,,44
7,Nov 8 2022 11:46AM,250307,10,"Nextsource Biotechnology, LLC",0.0,,,2
8,Nov 8 2022 11:46AM,250306,10,ISDIN Corporation,0.0,,,22
9,Nov 8 2022 11:44AM,250305,10,ISDIN Corporation,0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 12:27PM,250315,10,"Citieffe, Inc.",2,,
1,Nov 8 2022 12:25PM,250314,16,Sartorius Bioprocess Solutions,1,,
2,Nov 8 2022 12:12PM,250308,10,Bio-PRF,11,,
3,Nov 8 2022 12:07PM,250313,10,Emerginnova,1,,
4,Nov 8 2022 12:07PM,250310,19,"AdvaGen Pharma, Ltd",20,1,
5,Nov 8 2022 12:04PM,250311,16,TASA USA Inc.,1,,
6,Nov 8 2022 11:56AM,250280,15,"Alliance Pharma, Inc.",44,,
7,Nov 8 2022 11:46AM,250307,10,"Nextsource Biotechnology, LLC",2,,
8,Nov 8 2022 11:46AM,250306,10,ISDIN Corporation,22,,
9,Nov 8 2022 11:44AM,250305,10,ISDIN Corporation,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 12:27PM,250315,10,"Citieffe, Inc.",2,,
1,Nov 8 2022 12:25PM,250314,16,Sartorius Bioprocess Solutions,1,,
2,Nov 8 2022 12:12PM,250308,10,Bio-PRF,11,,
3,Nov 8 2022 12:07PM,250313,10,Emerginnova,1,,
4,Nov 8 2022 12:07PM,250310,19,"AdvaGen Pharma, Ltd",20,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 12:27PM,250315,10,"Citieffe, Inc.",2.0,NaN,NaN
1,Nov 8 2022 12:25PM,250314,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
2,Nov 8 2022 12:12PM,250308,10,Bio-PRF,11.0,NaN,NaN
3,Nov 8 2022 12:07PM,250313,10,Emerginnova,1.0,NaN,NaN
4,Nov 8 2022 12:07PM,250310,19,"AdvaGen Pharma, Ltd",20.0,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 12:27PM,250315,10,"Citieffe, Inc.",2.0,0.0,0.0
1,Nov 8 2022 12:25PM,250314,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
2,Nov 8 2022 12:12PM,250308,10,Bio-PRF,11.0,0.0,0.0
3,Nov 8 2022 12:07PM,250313,10,Emerginnova,1.0,0.0,0.0
4,Nov 8 2022 12:07PM,250310,19,"AdvaGen Pharma, Ltd",20.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4004658,61.0,12.0,0.0
12,250262,1.0,0.0,0.0
15,750610,84.0,89.0,0.0
16,500625,2.0,0.0,0.0
19,1001195,22.0,7.0,0.0
20,2252530,16.0,7.0,15.0
21,1000897,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4004658,61.0,12.0,0.0
1,12,250262,1.0,0.0,0.0
2,15,750610,84.0,89.0,0.0
3,16,500625,2.0,0.0,0.0
4,19,1001195,22.0,7.0,0.0
5,20,2252530,16.0,7.0,15.0
6,21,1000897,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,61.0,12.0,0.0
1,12,1.0,0.0,0.0
2,15,84.0,89.0,0.0
3,16,2.0,0.0,0.0
4,19,22.0,7.0,0.0
5,20,16.0,7.0,15.0
6,21,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,61.0
1,12,Released,1.0
2,15,Released,84.0
3,16,Released,2.0
4,19,Released,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,15.0,0.0
Executing,12.0,0.0,89.0,0.0,7.0,7.0,0.0
Released,61.0,1.0,84.0,2.0,22.0,16.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,15.0,0.0
1,Executing,12.0,0.0,89.0,0.0,7.0,7.0,0.0
2,Released,61.0,1.0,84.0,2.0,22.0,16.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,15.0,0.0
1,Executing,12.0,0.0,89.0,0.0,7.0,7.0,0.0
2,Released,61.0,1.0,84.0,2.0,22.0,16.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()